# Modelo Final

In [11]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None

import sys
sys.path.append("../../")
from src import funciones as fun
from src import variables as var

Observamos las métricas de todos los modelos realizados hasta el moemnto para decidir sobre cual es mejor centrar nuestros esfuerzos de mejora:

In [6]:
df_metricas = pd.read_csv('../archivos/metricas_reg_logística.csv', index_col = 0)
df_metricas

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.767380,0.750643,0.791328,0.770449,0.535009,test,RLog balanceada
1,0.768227,0.749536,0.808000,0.777671,0.536328,train,RLog balanceada
0,0.788202,0.623377,0.513369,0.563050,0.424998,test,RLog desbalanceada
1,0.805156,0.658711,0.553846,0.601744,0.474047,train,RLog desbalanceada
0,0.719972,0.474747,0.502674,0.488312,0.295769,test,D.Tree desbalanceado I
1,0.996800,0.998650,0.989298,0.993952,0.991776,train,D.Tree desbalanceado I
0,0.687166,0.697947,0.644986,0.670423,0.373593,test,D.Tree balanceado I
1,0.997324,1.000000,0.994667,0.997326,0.994649,train,D.Tree balanceado I
0,0.787491,0.662338,0.409091,0.505785,0.379918,test,R.Forest desbalanceado I
1,0.817067,0.724903,0.502341,0.593441,0.480384,train,R.Forest desbalanceado I


- Como conclusión general podemos intuir que de manera generalizada, los modelos están funcionando mejor con la variable respuesta balanceada.
- Dentro de los datos balanceados, los modelos que mejor parecen funcionar son el el de Regresión Logística (*RLog balanceada*) y el Random Forest (*R.Forest balanceado I*).
    - Aunque inicialmente parece mejor modelo el de Regresión Logística, consideramos que podría resultar más sencillo mejorar el modelo Random Forest inicialmente manejando los hiperparámetros.
        - No obstante, si contáramos como next steps nos plantearíamos centrarnos en afinar el enconding de nuestras variables categoricas y balancear los datos probando diferentes métodos de cara a intentar mejorar los resultados del modelo de regresión.

Procedemos a afinar el modelo de Random Forest:

In [12]:
# empleamos el dataframe con los datos balanceados:
df = pd.read_pickle('../archivos/balanceado.pkl')

In [13]:
df.head()

,tenure,monthlycharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,phoneservice_No,phoneservice_Yes,multiplelines_No,multiplelines_No phone service,multiplelines_Yes,streamingtv_No,streamingtv_No internet service,streamingtv_Yes,paperlessbilling_No,paperlessbilling_Yes,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,contract,paymentmethod,churn
0,1.571829,1.020541,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0
1,1.245878,0.515286,1,0,1,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0
2,0.838438,0.952398,1,0,0,1,1,0,0,1,1,0,0,0,0,1,0,1,1,1,2,1,0,1,2,0
3,-0.832064,0.503652,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,1,1,1,2,1,1,2,1,0
4,-0.750577,-1.510719,0,1,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0


In [31]:
X = df.drop("churn", axis = 1)
y = df["churn"]

In [32]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [33]:
param2 = {"max_depth": [8, 10, 12, 14],
        "max_features": [4, 5, 6, 7],
        "min_samples_split": [5, 6, 7],
        "min_samples_leaf": [40 ,50, 60]} 

In [34]:
gs_rf2 = GridSearchCV(
            estimator=RandomForestClassifier(random_state=42),
            param_grid= param2, 
            cv=10, 
            verbose=-1) 

In [35]:
gs_rf2.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [8, 10, 12, 14],
                         'max_features': [4, 5, 6, 7],
                         'min_samples_leaf': [40, 50, 60],
                         'min_samples_split': [5, 6, 7]},
             verbose=-1)

In [27]:
bosque2 = gs_rf2.best_estimator_
bosque2

RandomForestClassifier(max_depth=10, max_features=5, min_samples_leaf=50,
                       min_samples_split=5, random_state=42)

In [23]:
# bosqueIII = bosque
# bosqueIII

RandomForestClassifier(max_depth=10, max_features=5, min_samples_leaf=50,
                       min_samples_split=3, random_state=42)

In [28]:
# bosqueIV = bosque
# bosqueIV

RandomForestClassifier(max_depth=10, max_features=5, min_samples_leaf=50,
                       min_samples_split=5, random_state=42)

In [29]:
y_pred_test_rf = bosque2.predict(x_test)
y_pred_train_rf = bosque2.predict(x_train)

In [21]:
# dt_results = fun.metricas(y_test, y_pred_test_rf,y_train, y_pred_train_rf, "Random Forest balanceado III")
# dt_results

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.756684,0.737913,0.785908,0.761155,0.513699,test,Random Forest balanceado III
1,0.768896,0.751711,0.805333,0.777599,0.537677,train,Random Forest balanceado III


In [30]:
dt_results2 = fun.metricas(y_test, y_pred_test_rf,y_train, y_pred_train_rf, "Random Forest balanceado IV")
dt_results2

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.756684,0.737913,0.785908,0.761155,0.513699,test,Random Forest balanceado IV
1,0.768896,0.751711,0.805333,0.777599,0.537677,train,Random Forest balanceado IV
